# Read Dataset

In [1]:
import pandas as pd

data = pd.read_excel("data/diy/sample_data_1.xlsx")
data.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1924499205722084096,Mon May 19 16:15:40 +0000 2025,0,Palang Pintu Pelintasan Terbuka Saat KA Maliob...,1924499205722084096,https://pbs.twimg.com/media/GrUx1A-XIAAwSH_.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/192449920572208...,118371948,NaN
1,1924455275739693056,Mon May 19 13:21:07 +0000 2025,6,ASN Pemkab Magetan Jadi Korban Meninggal Kecel...,1924455275739693056,https://pbs.twimg.com/media/GrUJ3-rWoAA0y4T.jpg,NaN,in,NaN,1,0,0,https://x.com/undefined/status/192445527573969...,118371948,NaN
2,1924350383109165056,Mon May 19 12:19:00 +0000 2025,0,@regar_op0sisi Harusnya di panggilin damkar in...,1924439646890820096,NaN,regar_op0sisi,in,NaN,0,0,0,https://x.com/undefined/status/192443964689082...,280381391,NaN
3,1924388809271034112,Mon May 19 08:57:00 +0000 2025,1,Kronologi Kecelakaan Kereta di Magetan Palang ...,1924388809271034112,https://pbs.twimg.com/media/GrTNbIsW0AA_PCC.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/192438880927103...,118371948,NaN
4,1924384204156026880,Mon May 19 08:38:42 +0000 2025,1,Kecelakaan KA Malioboro Ekspres di Magetan Tew...,1924384204156026880,https://pbs.twimg.com/media/GrTJPAyW0AAZg7J.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/192438420415602...,118371948,NaN


In [2]:
# Contoh: styling kolom agar wrap
def display_fullscreen_wrap(df):
    return df.style.set_properties(
        **{
            "white-space": "pre-wrap",  # wrap isi cell
            "word-break": "break-word",  # pecah di mana saja kalau kepanjangan
            "width": "900px",  # bisa diganti sesuai kebutuhan
            "max-width": "700px",  # atur lebar kolom maksimal
        }
    )

# EDA

In [3]:
data.columns

Index(['conversation_id_str', 'created_at', 'favorite_count', 'full_text',
       'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location',
       'quote_count', 'reply_count', 'retweet_count', 'tweet_url',
       'user_id_str', 'username'],
      dtype='object')

### Summary

In [4]:
info = data.info()
print("Summary :", info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7744 entries, 0 to 7743
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   conversation_id_str      7744 non-null   int64  
 1   created_at               7744 non-null   object 
 2   favorite_count           7744 non-null   int64  
 3   full_text                7744 non-null   object 
 4   id_str                   7744 non-null   int64  
 5   image_url                1567 non-null   object 
 6   in_reply_to_screen_name  3735 non-null   object 
 7   lang                     7744 non-null   object 
 8   location                 0 non-null      float64
 9   quote_count              7744 non-null   int64  
 10  reply_count              7744 non-null   int64  
 11  retweet_count            7744 non-null   int64  
 12  tweet_url                7744 non-null   object 
 13  user_id_str              7744 non-null   int64  
 14  username                

### Find Missing Values

In [5]:
# menghitung dan menampilkan missing values
print("Jumlah missing values disetiap kolom:\n", data.isnull().sum())

Jumlah missing values disetiap kolom:
 conversation_id_str           0
created_at                    0
favorite_count                0
full_text                     0
id_str                        0
image_url                  6177
in_reply_to_screen_name    4009
lang                          0
location                   7744
quote_count                   0
reply_count                   0
retweet_count                 0
tweet_url                     0
user_id_str                   0
username                   7744
dtype: int64


### Delete unnecessary columns and does not provide any information

In [6]:
data = data[["full_text"]]
data.head()

,full_text
0,Palang Pintu Pelintasan Terbuka Saat KA Maliob...
1,ASN Pemkab Magetan Jadi Korban Meninggal Kecel...
2,@regar_op0sisi Harusnya di panggilin damkar in...
3,Kronologi Kecelakaan Kereta di Magetan Palang ...
4,Kecelakaan KA Malioboro Ekspres di Magetan Tew...


### Check Duplicate Data that contain in the content column

In [7]:
data.duplicated().sum()

np.int64(838)

# Data Preparation

### Case Folding

In [8]:
# merubah jenis huruf menjadi huruf kecil
data["full_text"] = data["full_text"].str.lower()
data.head()

,full_text
0,palang pintu pelintasan terbuka saat ka maliob...
1,asn pemkab magetan jadi korban meninggal kecel...
2,@regar_op0sisi harusnya di panggilin damkar in...
3,kronologi kecelakaan kereta di magetan palang ...
4,kecelakaan ka malioboro ekspres di magetan tew...


## Cleaning Data

### Emoji To Word

In [9]:
import pandas as pd
from indoNLP.preprocessing import (
    pipeline,
    replace_word_elongation,
    replace_slang,
    remove_html,
    remove_url,
    emoji_to_words
)
# Apply emoji_to_words to the text column
data["full_text"] = data["full_text"].apply(lambda x: emoji_to_words(str(x), lang="id"))
display_fullscreen_wrap(data.head())

,full_text
0,palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari https://t.co/2rau13j0cp https://t.co/ouql7ahsug
1,asn pemkab magetan jadi korban meninggal kecelakaan ka malioboro ekspres https://t.co/hin963viyt https://t.co/b7bvzvythg
2,@regar_op0sisi harusnya di panggilin damkar ini.. gercep langsung ke lokasi mengamankan kecelakaan
3,kronologi kecelakaan kereta di magetan palang pintu diduga terbuka saat ka malioboro ekspres melintas https://t.co/f3sgxfpbfv https://t.co/7cog3mw3q2
4,kecelakaan ka malioboro ekspres di magetan tewaskan 4 orang ini kronologinya https://t.co/azsah90363 https://t.co/cgjchkd0tu


### Remove HTML

In [10]:
# data["full_text"] = data["full_text"].apply(lambda x: remove_html(str(x)))
# display_fullscreen_wrap(data.head())

### Remove URL

In [11]:
# data["full_text"] = data["full_text"].apply(lambda x: remove_url(str(x)))
# display_fullscreen_wrap(data.head())

### Replace Slang

In [12]:
# data["full_text"] = data["full_text"].apply(lambda x: replace_slang(str(x)))
# display_fullscreen_wrap(data.head())

### Replace Word Elongation

In [13]:
data["full_text"] = data["full_text"].apply(lambda x: replace_word_elongation(str(x)))
display_fullscreen_wrap(data.head())

,full_text
0,palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari https://t.co/2rau13j0cp https://t.co/ouql7ahsug
1,asn pemkab magetan jadi korban meninggal kecelakaan ka malioboro ekspres https://t.co/hin963viyt https://t.co/b7bvzvythg
2,@regar_op0sisi harusnya di panggilin damkar ini.. gercep langsung ke lokasi mengamankan kecelakaan
3,kronologi kecelakaan kereta di magetan palang pintu diduga terbuka saat ka malioboro ekspres melintas https://t.co/f3sgxfpbfv https://t.co/7cog3mw3q2
4,kecelakaan ka malioboro ekspres di magetan tewaskan 4 orang ini kronologinya https://t.co/azsah90363 https://t.co/cgjchkd0tu


### Replace User Mentions

In [14]:
import re

# Function to replace URLs and user mentions
def replace_urls_and_mentions(text):
    # Replace URLs with HTTPURL
    text = re.sub(
        r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        "HTTPURL",
        text,
    )
    text = re.sub(
        r"www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        "HTTPURL",
        text,
    )

    # Replace user mentions with @USER
    text = re.sub(r"@[A-Za-z0-9_]+", "@USER", text)

    return text

# Apply URL and mention replacement
data["full_text"] = data["full_text"].apply(lambda x: replace_urls_and_mentions(str(x)))
display_fullscreen_wrap(data.head())

,full_text
0,palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari HTTPURL HTTPURL
1,asn pemkab magetan jadi korban meninggal kecelakaan ka malioboro ekspres HTTPURL HTTPURL
2,@USER harusnya di panggilin damkar ini.. gercep langsung ke lokasi mengamankan kecelakaan
3,kronologi kecelakaan kereta di magetan palang pintu diduga terbuka saat ka malioboro ekspres melintas HTTPURL HTTPURL
4,kecelakaan ka malioboro ekspres di magetan tewaskan 4 orang ini kronologinya HTTPURL HTTPURL
